# IXS/ETH Liquidity Migration: Client Synthesis

**Prerequisites:** Run `ixs_execution_quality.ipynb` (D1) and `ixs_vault_performance.ipynb` (D2) first — they export the CSVs loaded below.

In [ ]:
import pandas as pd

slip = pd.read_csv('data/slippage_summary.csv')
vault_df = pd.read_csv('data/vault_summary.csv')

def vm(metric):
    """Look up a vault metric by name."""
    row = vault_df[vault_df['metric'] == metric]
    return row['value'].values[0] if len(row) > 0 else None

buy_slip = slip[slip['direction'] == 'ETH→IXS'].reset_index(drop=True)
sell_slip = slip[slip['direction'] == 'IXS→ETH'].reset_index(drop=True)

# Parse vault numbers upfront
vault_ret = float(vm('vault_return_pct'))
hodl_ret = float(vm('hodl_return_pct'))
fr_ret = float(vm('fullrange_return_pct'))
vault_ann = float(vm('vault_ann_pct'))
hodl_ann = float(vm('hodl_ann_pct'))
fr_ann = float(vm('fullrange_ann_pct'))
mgmt_pp = float(vm('mgmt_premium_ann_pp'))
days = float(vm('days'))
gas_cost = float(vm('gas_cost_usd')) if vm('gas_cost_usd') is not None else 0.0
n_rebalances = int(float(vm('n_rebalances'))) if vm('n_rebalances') is not None else 0

print(f"Data loaded: {vm('period_start')} to {vm('period_end')} ({int(days)} days)")

## Bottom Line

Two questions matter:

1. **Are traders getting a better deal?** (Execution quality: V2 vs V4)
2. **Is Arrakis making more money than the client could on their own?** (Vault performance vs passive LP)

In [ ]:
# Bottom-line numbers
print("=" * 60)
print(f"  Vault delivered {vault_ret - hodl_ret:+.1f}pp over HODL")
print(f"  and {vault_ret - fr_ret:+.1f}pp over passive LP in {int(days)} days.")
print(f"  Management premium: {mgmt_pp:+.2f}pp annualized.")
if gas_cost > 0:
    print(f"  Rebalancing gas cost: ${gas_cost:,.0f} ({n_rebalances} rebalances)")
print("=" * 60)

In [ ]:
# Q1: Execution quality — gross slippage (price impact)
print("Q1: EXECUTION QUALITY")
print("=" * 75)
print(f"\n{'Size':<10} {'Direction':<12} {'V2 Gross':>10} {'V4 Gross':>10} {'Improvement':>14} {'Cap Eff':>10}")
print("-" * 75)
for _, row in slip.iterrows():
    imp = row['gross_improvement_pct']
    ce = row['capital_efficiency']
    print(f"${row['size_usd']:>7,.0f}  {row['direction']:<12} {row['v2_gross_avg']:>9.4f}% {row['v4_gross_avg']:>9.4f}% "
          f"{imp:>+12.1f}% {ce:>9.1f}x")

avg_cap_eff = slip['capital_efficiency'].mean()
print(f"\nAvg capital efficiency: {avg_cap_eff:.1f}x")

In [ ]:
# Q1 continued: Net cost (gross + fee)
print("\nNET COST TO TRADER: V2 (0.30% fee) vs V4 (0.70% fee)")
print("=" * 75)
print(f"{'Size':<10} {'Direction':<12} {'V2 Net':>10} {'V4 Net':>10} {'Δ (V2-V4)':>12} {'Cheaper':>10}")
print("-" * 75)
for _, row in slip.iterrows():
    delta = row['v2_net_avg'] - row['v4_net_avg']
    winner = "V4" if delta > 0 else "V2"
    print(f"${row['size_usd']:>7,.0f}  {row['direction']:<12} {row['v2_net_avg']:>9.4f}% {row['v4_net_avg']:>9.4f}% "
          f"{delta:>+10.4f}pp  {winner:>8}")

# Break-even
for label, df in [("ETH→IXS", buy_slip), ("IXS→ETH", sell_slip)]:
    be_val = df['breakeven_usd'].iloc[0]
    if pd.notna(be_val):
        print(f"\n  {label} break-even: ~${float(be_val):,.0f}")
    elif df['v4_net_avg'].iloc[-1] < df['v2_net_avg'].iloc[-1]:
        print(f"\n  {label}: V4 cheaper at all tested sizes")
    else:
        print(f"\n  {label}: V2 cheaper at all tested sizes")

## Q2: Vault Performance

In [ ]:
# Q2: Vault performance vs benchmarks
print("Q2: VAULT PERFORMANCE")
print("=" * 55)
print(f"{'Strategy':<20} {'Total Return':>14} {'Annualized':>14}")
print("-" * 55)
print(f"{'Arrakis Vault':<20} {vault_ret:>+13.2f}% {vault_ann:>+13.2f}%")
print(f"{'HODL':<20} {hodl_ret:>+13.2f}% {hodl_ann:>+13.2f}%")
print(f"{'Full-Range LP':<20} {fr_ret:>+13.2f}% {fr_ann:>+13.2f}%")

print(f"\nManagement premium: {mgmt_pp:+.2f}pp annualized")
print(f"Observation period: {int(days)} days")

print(f"\nReturn decomposition:")
print(f"  Price return:      ${float(vm('price_return_usd')):+,.2f}")
print(f"  IL (full-range):   ${float(vm('il_fullrange_usd')):+,.2f}")
print(f"  Mgmt premium:      ${float(vm('mgmt_premium_usd')):+,.2f}")
if gas_cost > 0:
    print(f"  Gas cost:          ${gas_cost:,.2f} ({n_rebalances} rebalances)")

## Tradeoffs

**What Arrakis does that passive LP can't:**
1. Concentrates liquidity around the current price — same TVL, deeper depth
2. Rebalances ranges as price moves — avoids out-of-range dead capital
3. Higher fee tier (0.70%) captures 2.3x more revenue per trade than V2's 0.30%

**What can go wrong:**
1. Concentrated positions amplify IL — if price moves fast, losses are larger than full-range
2. Manager dependency — vault performance depends entirely on Arrakis's strategy quality
3. Smart contract risk — additional vault layer on top of UniV4

## Recommendation

**Migrate remaining V2 funds.** The case rests on LP performance, not trader costs:
- The vault outperformed both HODL and passive full-range LP during a declining market
- Active rebalancing preserved value that passive strategies lost to IL
- The higher fee tier generates more revenue per dollar of TVL
- Consolidating depth into one venue eliminates split-routing

**Caveat:** ~2.5 months of data. This outperformance occurred during IXS price decline — where active management has its largest advantage. Performance in sideways or rallying markets may differ.

## Client-Facing Summary

On December 10, 2025, IXS/ETH liquidity was migrated from UniswapV2 to an Arrakis-managed UniswapV4 vault.

**Is Arrakis making money the client couldn't make on their own?** Yes. The managed vault outperformed both doing-nothing (HODL) and passive full-range LP during a period of IXS price decline. Arrakis's active rebalancing kept liquidity concentrated near the current price, avoiding the worst impermanent loss scenarios. The management premium is positive and significant.

**Are traders getting a better deal?** Mixed. V4's concentrated liquidity reduces price impact, but the higher fee (0.70% vs 0.30%) means net cost is higher for small-medium trades. For the LP, this is a feature: more fee revenue per trade.

**Should remaining V2 funds migrate?** The data supports it — on LP performance grounds. The observation period is short (~2.5 months, during a price decline), so continued monitoring is warranted. But the early signal is clear: active management is delivering value.

## Methodology

All data via RPC calls to Ethereum mainnet (Alchemy archive node). No subgraphs or external APIs.

**D1 — Execution Quality:**
- V2 slippage: `getReserves()` + constant-product formula. 1 RPC/block, all sizes computed locally.
- V4 slippage: `StateView.getSlot0()` for spot + `Quoter.quoteExactInputSingle()` for execution price.
- Net slippage = gross (price impact) + fee. Break-even via linear interpolation.

**D2 — Vault Performance:**
- Vault tracking: `totalUnderlying()` at daily block intervals.
- Prices: ETH/USD (Chainlink), IXS/ETH (V4 `sqrtPriceX96`).
- HODL benchmark: initial amounts x current prices.
- Full-range LP: $V = V_{\text{hodl}} \times 2\sqrt{r}/(1+r)$.
- Decomposition: vault = HODL + IL + management premium.
- Gas cost: estimated from `baseFeePerGas` at rebalance blocks x avg gas per rebalance.

**On management premium:** This metric captures the total value-add from active management — both fee revenue earned by concentrated positions and rebalancing alpha. Decomposing these two components would require scanning all swap events on-chain (infeasible at scale with standard RPC). The total is what matters: vault outperformance over passive LP.